In [1]:
import pycaret
import ee
import numpy as np
import pandas as pd
import geemap

In [2]:
# ee.Authenticate()
ee.Initialize()

Enter verification code: 4/1AX4XfWiSaZAJ-3IziNfuPDmjQkIgORs0tI_Yq045SyPgm7JOtzPbM8LfPj4

Successfully saved authorization token.


In [3]:
Map = geemap.Map()
Map

Map(center=[40, -100], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(T…

In [4]:
grid = ee.FeatureCollection('users/geethensingh/WTWTA_grid75457')
print(grid.size().getInfo())
Map.centerObject(grid)
Map.addLayer(grid)

grid_img = grid.reduceToImage(**{'properties': ['fid'], 'reducer': ee.Reducer.first()}).rename('fid')
Map.addLayer(grid_img)

6


In [5]:
label = ee.ImageCollection.fromImages([ee.Image('users/jdmwhite/UD_buf_mean'),ee.Image('users/geethensingh/UD_buf_GPS')]).mean().rename('label')
covs = ee.Image('users/jdmwhite/WTWTA_covariates_1000_sel').addBands([grid_img, label])
# covs = ee.Image('users/jdmwhite/WTWTA_covariates_1000_sel').addBands([label])
# print(covs.bandNames().getInfo())
pts = ee.FeatureCollection('users/jdmwhite/buff_sampled_points').select([])
print(pts.first().getInfo())

{'type': 'Feature', 'geometry': {'type': 'Point', 'coordinates': [31.758101185332148, -24.595835104973677]}, 'id': '00000000000000001d2b', 'properties': {}}


In [6]:
print(pts.size().getInfo())
extracted_covs = covs.reduceRegions(**{'collection': pts, 'scale': 1000, 'reducer': ee.Reducer.first()})
print(extracted_covs.size().getInfo())

10491
10491


In [7]:
def largefc_to_pandas(featurecollection, step_size = 5000):
    """
    function to convert a featurecollection with more than 5000 elements to a pandas dataframe
    
    featurecollection: earth engine feature collection with more than 5000 elements
    step_size: Th enumber of elemenst to process at once. Must be less than equal to 5000
    
    """
    df = pd.DataFrame()
    fclist = featurecollection.toList(featurecollection.size())
    fcsize = featurecollection.size().getInfo()
    for i in range(0,fcsize,step_size):
        fc = ee.FeatureCollection(fclist.slice(i,i+step_size))
        df = df.append(geemap.ee_to_pandas(fc))
    return df

In [8]:
df  = largefc_to_pandas(extracted_covs)
print(f"There are {len(df)} features")
print(df.info(memory_usage='deep'))
print(df.head(5))

There are 10491 features
<class 'pandas.core.frame.DataFrame'>
Int64Index: 10491 entries, 0 to 490
Data columns (total 47 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   b1      10116 non-null  float64
 1   b10     10116 non-null  float64
 2   b11     10116 non-null  float64
 3   b12     10116 non-null  float64
 4   b13     10116 non-null  float64
 5   b14     10116 non-null  float64
 6   b15     10116 non-null  float64
 7   b16     10116 non-null  float64
 8   b17     10116 non-null  float64
 9   b18     10116 non-null  float64
 10  b19     10116 non-null  float64
 11  b2      10116 non-null  float64
 12  b20     10116 non-null  float64
 13  b21     10116 non-null  float64
 14  b22     10027 non-null  float64
 15  b23     10116 non-null  float64
 16  b24     10116 non-null  float64
 17  b25     10116 non-null  float64
 18  b26     10116 non-null  float64
 19  b27     10116 non-null  float64
 20  b28     10116 non-null  float64
 21  b3      1011

In [9]:
df_sub = df[['label','fid','b1','b10','b11','b12','b13','b14','b15','b16','b17','b18','b19','b2','b20','b21', 'b22', 'b23', 'b24', 'b25', 'b26','b27','b28','b3','b32','b33','b34','b35','b36','b37','b38','b4','b43','b44','b45','b5','b6','b7','b8','b9','b29','b42']]

In [10]:
df_sub.shape

(10491, 42)

In [11]:
df_sub.dtypes

label    float64
fid        int64
b1       float64
b10      float64
b11      float64
b12      float64
b13      float64
b14      float64
b15      float64
b16      float64
b17      float64
b18      float64
b19      float64
b2       float64
b20      float64
b21      float64
b22      float64
b23      float64
b24      float64
b25      float64
b26      float64
b27      float64
b28      float64
b3       float64
b32      float64
b33      float64
b34      float64
b35      float64
b36      float64
b37      float64
b38      float64
b4       float64
b43      float64
b44      float64
b45      float64
b5       float64
b6       float64
b7       float64
b8       float64
b9       float64
b29      float64
b42      float64
dtype: object

In [12]:
df_sub = df_sub.fillna(0)

In [13]:
df_sub.isna().sum()

label    0
fid      0
b1       0
b10      0
b11      0
b12      0
b13      0
b14      0
b15      0
b16      0
b17      0
b18      0
b19      0
b2       0
b20      0
b21      0
b22      0
b23      0
b24      0
b25      0
b26      0
b27      0
b28      0
b3       0
b32      0
b33      0
b34      0
b35      0
b36      0
b37      0
b38      0
b4       0
b43      0
b44      0
b45      0
b5       0
b6       0
b7       0
b8       0
b9       0
b29      0
b42      0
dtype: int64

In [ ]:
df_sub.drop(['fid'],axis=1)

In [30]:
from pycaret.regression import *
exp_reg101 = setup(data = df_sub.drop(['fid'],axis=1), target = 'label', fold_groups = pd.Series(df_sub.fid), fold_strategy = "groupkfold", imputation_type='simple', session_id=42)

,Description,Value
0,session_id,42
1,Target,label
2,Original Data,"(10491, 41)"
3,Missing Values,False
4,Numeric Features,40
5,Categorical Features,0
6,Ordinal Features,False
7,High Cardinality Features,False
8,High Cardinality Method,None
9,Transformed Train Set,"(7343, 39)"


In [33]:
exp_reg101

([('Setup Config',
                                  Description             Value
   0                               session_id                42
   1                                   Target             label
   2                            Original Data       (10491, 41)
   3                           Missing Values             False
   4                         Numeric Features                40
   5                     Categorical Features                 0
   6                         Ordinal Features             False
   7                High Cardinality Features             False
   8                  High Cardinality Method              None
   9                    Transformed Train Set        (7343, 39)
   10                    Transformed Test Set        (3148, 39)
   11                      Shuffle Train-Test              True
   12                     Stratify Train-Test             False
   13                          Fold Generator        GroupKFold
   14                

In [44]:
best = compare_models()

In [45]:
regression_results = pull()

In [46]:
print(regression_results)

Empty DataFrame
Columns: []
Index: []
